In [1]:
import sys

sys.path.insert(0, '..')

#!{sys.executable} -m pip install folium
#!{sys.executable} -m pip install openapi_core


In [2]:
from morpheus.project.types.discretization.spatial import Rotation, LengthUnit, Grid
from morpheus.project.types.geometry import Polygon

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96609040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(20)
length_unit = LengthUnit.meters()
relative_col_coordinates = [0, 0.1, 0.21, 0.22, 0.23, 0.24, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
relative_row_coordinates = [0, 0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 1]

grid = Grid.from_polygon_with_relative_coordinates(
  polygon=polygon,
  relative_col_coordinates=relative_col_coordinates,
  relative_row_coordinates=relative_row_coordinates,
  rotation=rotation,
)
polygon

Polygon(coordinates=[[(13.922514437551428, 50.964720483303836), (13.925250781947113, 50.965228748412386), (13.925036413951403, 50.96623732041704), (13.92222441026388, 50.96609040370362), (13.922514437551428, 50.964720483303836)]], type='Polygon')

In [3]:
grid.get_wgs_coordinates()

([13.92201083954258,
  13.922325948095247,
  13.922506010125344,
  13.922536020463694,
  13.922566030802043,
  13.922596041140393,
  13.922686072155441,
  13.92291114969306,
  13.923211253076555,
  13.923511356460047,
  13.923811459843542,
  13.924111563227036,
  13.924411666610526,
  13.92471176999402],
 [50.9658859127032,
  50.96574170338251,
  50.96572796913816,
  50.96571423488973,
  50.96570050063724,
  50.96568676638068,
  50.965645563586676,
  50.96554255644179,
  50.96540521322662,
  50.9652678696054,
  50.96513052557811,
  50.96485583630537])

In [4]:
grid.bbox()

(13.921781413845427, 50.96461542216845, 13.925576187734443, 50.96667675655004)

In [5]:
import folium

m = folium.Map(crs="EPSG3857", zoom_start=12, location=[50.965, 13.922])
m.add_child(folium.GeoJson(polygon.as_geojson()))
m.fit_bounds(m.get_bounds())
m

In [6]:
m.add_child(folium.GeoJson(grid.bounding_box_geometry().as_geojson()))
m.fit_bounds(m.get_bounds())
m

In [7]:

cell_geometries = grid.get_cell_geometries()

n_x = grid.n_cols()
n_y = grid.n_rows()

for x in range(n_x):
  for y in range(n_y):
    cell_geometry = cell_geometries[x][y]
    folium.GeoJson(cell_geometry.as_geojson()).add_to(m)

m

IndexError: list index out of range

In [ ]:
# Render GRid Rows and Columns
from morpheus.project.types.discretization.spatial import ActiveCells

cells = ActiveCells.from_polygon(polygon, grid)

m = folium.Map(zoom_start=12, location=[50.965, 13.922])
folium.GeoJson(polygon.as_geojson(), name="hello world").add_to(m)
folium.GeoJson(grid.to_geojson().as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())
m


In [ ]:
from morpheus.project.types.discretization.spatial import ActiveCells

cells = ActiveCells.from_polygon(polygon, grid)
cells_geometries = cells.to_geojson(grid)

m = folium.Map(zoom_start=12, location=[50.965, 13.922])
folium.GeoJson(polygon.as_geojson(), name="hello world").add_to(m)
folium.GeoJson(grid.bounding_box_geometry().as_geojson()).add_to(m)
folium.GeoJson(cells_geometries.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())
m

In [ ]:
cells_outline = cells.outline_to_geojson(grid)

m = folium.Map(zoom_start=12, location=[50.965, 13.922])
folium.GeoJson(polygon.as_geojson()).add_to(m)
folium.GeoJson(grid.bounding_box_geometry().as_geojson()).add_to(m)
folium.GeoJson(cells_outline.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())

m